# Unsupervised methods in NLP



### Themes:
- Part I: Analyzing topics (unsupervised NLP basics)
    - Term weighting with TF-IDF
    - Document clustering
- Part II: Transfer learning (modern NLP)
    - Word vectors (word2vec)
    - Pre-trained language models (ELMo, BERT)



## Overview

- Goal: identify meaningful relationships in raw text *internally*
    - No need for manual annotation (as in supervised learning)
- Unsupervised methods involve:
    - Counting frequencies of word occurence in different contexts
    - Counting/estimating frequencies of word co-occurrence in different contexts
    - Machine learning methods for clustering objects (e.g. documents)
    - Machine learning methods for modeling word sequences (language modeling)


# Part I: Analyzing topics (unsupervised NLP basics)

## Term weighting with TF-IDF

### Recap from "Basic NLP" lecture: _tf-idf weighting_

* TF = term frequency *tf(t, d)*, how many times the term _t_ appears in document *d*
* DF = document frequency *df(t)*, in how many documents the term *t* appears
* IDF = inverse document frequency, *m/df(t)*, where *m* is the total number of documents in your collection
* TF-IDF = **tf(t, d) * idf(t)**
    * Usually calculated using logaritmic (sublinear) scale --> tf(t, d) * log(idf(t))
    
* common in information retrieval, also used in document classification and clustering
* scale down the impact of tokens that occur very frequently in many documents and are hence empirically less informative/distinctive than words that occur in a small fraction of the documents

More reading on [tf-idf](https://nlp.stanford.edu/IR-book/html/htmledition/term-frequency-and-weighting-1.html) and [scaling and normalization variations](https://nlp.stanford.edu/IR-book/html/htmledition/variant-tf-idf-functions-1.html) (in the context of information retrieval).

In [ ]:
### Calculate TF-IDF scores on a corpus

import json
from sklearn.feature_extraction.text import TfidfVectorizer

with open("Data/patents.txt", "rt", encoding="utf-8") as f:
    documents = [line.strip() for line in f.readlines()]

# Set parameters and initialize
tfidf_vectorizer = TfidfVectorizer(min_df=2, use_idf=True, sublinear_tf=True, max_df=1.0, max_features=20000)
# Tip: the vectorizer also supports extracting n-gram features (common short sequences of words), which may be more descriptive but also much less frequent

# Calcualate term-document matrix with tf-idf scores
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

# Check matrix shape
tfidf_matrix.toarray().shape # N_docs x N_terms

In [ ]:
documents[0]

In [ ]:
# Inspect terms in vocabulary
print(tfidf_vectorizer.get_feature_names()[:10])
print(tfidf_vectorizer.get_feature_names()[-10:])

In [ ]:
## Inspect document frequencies (counts) of terms

from collections import Counter
terms_in_docs = tfidf_vectorizer.inverse_transform(tfidf_matrix)
token_counter = Counter()
for terms in terms_in_docs:
    token_counter.update(terms)

for term, count in token_counter.most_common(20):
    print("%d\t%s" % (count, term))

In [ ]:
# Alternatively: Inspect IDF values directly
#print(sorted(zip(features,tfidf_vectorizer._tfidf.idf_),key=lambda x:x[1])[:20])

In [ ]:
## Inspect top terms per document

features = tfidf_vectorizer.get_feature_names()
for doc_i in range(5):
    print("\nDocument %d, top terms by TF-IDF" % doc_i)
    for term, score in sorted(list(zip(features,tfidf_matrix.toarray()[doc_i])), key=lambda x:-x[1])[:5]:
        print("%.2f\t%s" % (score, term))

## Document clustering

- Clustering: grouping of similar objects in order to structure a set
    - E.g., clustering documents in a corpus -> reveal common topics and provide overview
- TF-IDF weighting for extracting meaningful *features* of documents, highlighting descriptive terms
- Goal: Given features, calculate similarities between documents and assign them into clusters

- The term-document matrix (`tfidf_matrix`) represents the corpus as tf-idf weights (features)
    - Each document represented as a (row) vector
    - Document vectors are sparse:
        - Dimensionality equal to the number of terms extracted from the corpus
        - Non-zero values only for the fraction of terms in document

In [ ]:
print(tfidf_matrix.toarray())

In [ ]:
print("Document vector length:", tfidf_matrix.shape[1])
for i in range(5):
    print("Non-zero dimensions for document %d: %d" % (i, len([x for x in tfidf_matrix.toarray()[i] if x > 0])))

In [ ]:
print("Sample word:", features[1000])
print("Occurs in %d documents" % len([x for x in tfidf_matrix.toarray()[:][1000] if x > 0]))
print("out of %d documents" % len(tfidf_matrix.toarray()))

### Vector space model

- A vector space model encodes meaning as coordinates in a high-dimensional space
    - E.g., tf-idf term vectors describe the topic of a document
- Semantic similarities are measured as distances in the space:
    - For vectors *x* and _y_:
        - Euclidean distance: $$d = \sqrt{ (x_1 - y_1)^2 + (x_2 - y_2)^2 + ... + (x_n - y_n)^2 }$$
        - Cosine similarity (distance = 1-similarity): $$s = cos(\sigma) = \frac{x \cdot y}{|x||y|}$$

- We use vector space notion of semantic similarity to group documents into clusters
    - Find common topics, go from many documents to fewer clusters 

More reading on the [vector space model](https://nlp.stanford.edu/IR-book/html/htmledition/the-vector-space-model-for-scoring-1.html).

### k-means clustering

- We will cluster documents based on their tf-idf vectors 
- and the relatively simple and widely used *k-means* method
    - Takes as argument *k* number of clusters to be found
    - Uses a random initialization (i.e., results can vary between runs)
    - Works according to this pseudo code (from the [Mining Massive Datasets book](http://infolab.stanford.edu/~ullman/mmds/ch7.pdf)):

<code>Initially choose k points that are likely to be in different clusters;
Make these points the centroids of their clusters;
FOR each remaining point p DO 
    find the centroid to which p is closest;
    Add p to the cluster of that centroid;
    Adjust the centroid of that cluster to account for p;
END;</code>

In [ ]:
# We use a subset of the documents in clustering for faster calculation and easier interpretation of results
matrix_sample = tfidf_matrix[:1000]

In [ ]:
from sklearn.cluster import KMeans

# Do clustering
km = KMeans(n_clusters=30, random_state=123, verbose=0)
km.fit(matrix_sample)

In [ ]:
import heapq, numpy as np

# Custom function to print top keywords for each cluster
def print_clusters(matrix, clusters, n_keywords=10):
    for cluster in range(min(clusters), max(clusters)+1):
        cluster_docs = [i for i, c in enumerate(clusters) if c == cluster]
        print("Cluster: %d (%d docs)" % (cluster, len(cluster_docs)))
        
        # Keep scores for top n terms
        new_matrix = np.zeros((len(cluster_docs), matrix.shape[1]))
        for cluster_i, doc_vec in enumerate(matrix[cluster_docs].toarray()):
            for idx, score in heapq.nlargest(n_keywords, enumerate(doc_vec), key=lambda x:x[1]):
                new_matrix[cluster_i][idx] = score

        # Aggregate scores for kept top terms
        keywords = heapq.nlargest(n_keywords, zip(new_matrix.sum(axis=0), features))
        print(', '.join([w for s,w in keywords]))
        print()


In [ ]:
print_clusters(matrix_sample, km.labels_)

In [ ]:
### Hierarchical clustering (alternative approach)

from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

Z = linkage(matrix_sample.todense(), metric='cosine', method='complete')
_ = dendrogram(Z, no_labels=True) # Plot dentrogram chart

In [ ]:
# View dendrogram for subset
Z_ = linkage(matrix_sample.todense()[:250], metric='cosine', method='complete')
_ = dendrogram(Z_, no_labels=True) # Plot dentrogram chart

In [ ]:
## Get flat clusters from cluster hierarchy

#clusters = fcluster(Z, 50, criterion='maxclust') # Create fix number of flat clusters
clusters = fcluster(Z, 0.99, criterion='distance') # Create flat clusters by distance threshold

print_clusters(matrix_sample, clusters)

### Probabilistic topic modeling

- Various machine learning methods for *topic modeling* extend the idea of topic clustering
- The most popular: [Latent Dirichlet Allocation (LDA) method](https://dl.acm.org/doi/pdf/10.1145/2133806.2133826) (see also [gensim library](https://radimrehurek.com/gensim/models/ldamodel.html))
- LDA performs a type of fuzzy clustering into *k* topics:

- 
    - A document can be assigned to *serveral topics*, according to a probability distribution
        - E.g., 68% Topic 6, 21% Topic 5, 11% Topic 2

-     - A topic is described by a *weighting of terms* (probability distribution)
        - The terms reflect term frequencies in documents assigned to the topic
        - The top-10 are often inspected, after some filtering/reranking

In [ ]:
## Topic modeling demo
#!pip3 install gensim

# Fast and simple tokenization
new_vectorizer = TfidfVectorizer()
word_tokenizer = new_vectorizer.build_tokenizer()
tokenized_text = [word_tokenizer(doc) for doc in documents]

# Train LDA model
from gensim import corpora, models
dictionary = corpora.Dictionary(tokenized_text)
lda_corpus = [dictionary.doc2bow(text) for text in tokenized_text]
lda_model = models.LdaModel(lda_corpus, id2word=dictionary, num_topics=10)

In [ ]:
# Inspect topics
for i, topic in lda_model.show_topics(num_words=10, formatted=False):
    print("Topic", i)
    for term, score in topic:
        print("%.4f\t%s" % (score,term))
    print()

In [ ]:
# Inspect topics
for i, topic in lda_model.show_topics(num_words=50, formatted=False):
    print("Topic", i)
    printed_terms = 0
    for term, score in topic:
        if printed_terms >= 10:
            break
        elif term in "the of and to for in or The is be may an a with at are on by as from can".split():
            continue
        printed_terms += 1
        print("%.4f\t%s" % (score,term))
    print()

# Part II: Transfer learning (modern NLP)

# Word vectors

- Recap: We used the term-document matrix to understand semantic similarities *between documents* based on overlapping keywords
- Now, we study similarities *between words*, by comparing the close contexts in which they appear
- The comparison is also done in a vector space, see illustration (with only 3 dimensions):

![semantic_space](figs/semspace.svg "Illustration of words embedded in a semantic vector space")

**Simple demo, document context:**
- We can also inspect the term-document matrix vertically, by extracting *column vectors* for terms
- Meaning by context: A term is described by the documents in which it occurs
- The tf-idf scores of the term vector (or word vector) define coordinates in a high-dimensional space
    - Distances can be measured, commonly by the cosine similarity function


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

print("Similar terms to:", features[3000])
# Get most similar terms according to the cosine similarity of their vectors (columns in the term-document matrix)
heapq.nlargest(10, zip(cosine_similarity(tfidf_matrix[:,3000].todense().T, tfidf_matrix.todense().T)[0], features))


### Better approach: Word co-occurrence matrix

- Look at a more immediate context of word use, e.g., +/- 5 words
- Count word co-occurrence statistics (--> word-word matrix)

**Analyzing word combinations (first-order co-occurrence):**
- Strongly associated pairs of words can be identified using the *pointwise mutual information* (PMI) score:
    - For words x and y: $$pmi(x,y) = \log \frac{p(x,y)}{p(x)p(y)} $$

    - where p(x,y) is estimated as the co-occurrence frequency and p(x) and p(y) as the independent occurence frequencies
- PMI can highlight common combinations of words and topics (e.g., "poor"-"service"; "handsome"-"man" / "beautiful"-"woman")

More reading on: [PMI](https://en.wikipedia.org/wiki/Pointwise_mutual_information) and [other association measures](http://collocations.de/AM/index.html).

**Analyzing word similarity (second-order co-occurrence):**

- "You shall know a word by the company it keeps" (J.R. Firth)
    - The meaning of a word can be understood by the distribution of words it tends to appear in (= distributional hypothesis)
    - Words may be similar although they to not directly co-occur (but share context words) --> 2nd order co-occurrence
- Cosine function between word vectors gives word pair similarity

- Vectors are very sparse: 
    - Vocabulary (dimensionality) grows with corpus size, while context size is small and fixed
    - Similarity can be estimated only when there is an overlap in terms
    - Many rare terms / sparse vectors --> very big corpora needed for good models --> huge memory usage and scalability issues

### Learning dense vectors

More useful vectors through:
- Matrix decomposition approach:
    - Reduce the number of dimensions os the semantic vector space, for instance, using Singular Value Decomposition (SVD)
        - Dimensionality reduction, e.g., from 10k or 100k to 100
    - Also used in Latent Semantic Analysis (LSA), applied to term-document matrix

- Neural network approach:
    - Learning *distributed representations* of concepts explored first by Hinton (1986)
    - word2vec, from 2013, is an important milestone in NLP:
        - Efficient estimation of dense word vectors, enabling fast training
        - Provides high precision vectors when applied to large corpora

- word2vec skip-gram model:
<img src="figs/skipgram_model.svg" alt="skip-gram model" width="800"/>

More reading on [counting vs. predicting word vectors](https://www.aclweb.org/anthology/P14-1023.pdf).

In [ ]:
### Train word vectors

import gensim # Make sure you also have cython installed to accelerate computation!

#import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Train word2vec model
vectors = gensim.models.Word2Vec(tokenized_text, size=100, window=5, min_count=5, workers=4)

In [ ]:
# Inspect a word vector
vectors.wv['process']

In [ ]:
# Inspect words with vectors most similar to a given word
print("Most similar to:", 'process')
print(vectors.wv.most_similar('process'))
print()

# ...or combination or words (average vector)
print("Most similar to:", ['process', 'payment'])
print(vectors.wv.most_similar(['process', 'payment']))

In [ ]:
# Inspect cosine similarities between specific words
print(vectors.wv.similarity('technology', 'infrastructure'))
print(vectors.wv.similarity('technology', 'system'))
print(vectors.wv.similarity('technology', 'consumer'))
print(vectors.wv.similarity('technology', 'advanced'))

In [ ]:
## Test for analogies and syntactic regularities (when trained on a large and suitable corpus)

"""
# Man is to King as Woman is to ...
# vec(woman)+vec(king)-vec(man)
print(vectors.most_similar(['woman','king'], negative=['man']))
print()

# Fall is to Fell as Watch is to ...
print(vectors.most_similar(['wear','fell'], negative=['fall']))
print()

print(vectors.most_similar(['Moscow','France'], negative=['Paris']))
print()

print(vectors.most_similar(['London','France'], negative=['Paris']))
"""

### Pre-trained word vectors
- Word vectors trained on large corpora achieve more accurate results
- For instance, word2vec vectors pre-trained on 100B words of Google News text can be [downloaded](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing) (1.5GB) and used in the same way as self-trained vectors
- word2vec is also efficient for training your own vectors on mid-sized corpora (e.g., 1B words)
    - Online demo [available here](http://bionlp-www.utu.fi/wv_demo/).

### Vectors as features

- Dense vectors are practical to use especially in neural network models for downstream tasks (e.g., text classification)
- Word vectors (often _word embeddings_) estimated on large amounts of raw text:
    - Embedding layer can serve as an abstraction of text input
    - Reduces reliance on (limited) annotated data in supervised tasks
    - Pre-trained word vectors can make NLP models more accurate and robust
    - This transfer of knowledge about language is called *transfer learning*

<img src="figs/simple_lstm.svg" alt="Neural network with word embeddings" width="400"/>



### Limitations

- Problem: Word vectors produced by word2vec are static with respect to the word
    - I.e., a term always has the same vector representation regardless of how it is used
    - Multiple senses of a word are not distinguished
    - Difficult to capture meaning of sentences (unless fine-tuned on some task)

In [ ]:
# Ambiguous word vector examples

print("Most similar to:", 'data')
print(vectors.wv.most_similar('data'))
print()

# Are the below average vectors less ambiguous?
print("Most similar to:", ['data','transfer'])
print(vectors.wv.most_similar(['data','transfer']))
print()

print("Most similar to:", ['data','storage'])
print(vectors.wv.most_similar(['data','storage']))
print()

## Contextualized word vectors with language modeling

- Word meaning can be disambiguated from the context in an instance of use
    - Compare:
        - "He wrote the **play** himself."
        - "He did **play** with the kids."
        
- Recent methods build on *language modeling* to provide *contextualized* word vectors

- Recap: A language model estimates the probability *P(x)* of seeing a word _x_ following a context, such as:

$$P(\ x\ |\ the\ students\ opened\ their\ )$$

![language modeling (source: mc.ai)](figs/language_modeling.png "Language modeling illustration")
The above suggestions would be deemed much more likely than, e.g., *P(x=keyboard)*, _P(x=follows)_ or *P(x=the)*.

Lanugage modeling recap: Consider a simple statistical model that estimates the probability of a word given a number of previous words in a text. For instance, given the words *the dog loved to*, the model would deem it much more likely that the next word is _play_ than any word that does not fit semantically or syntactically (e.g., *green*, _played_ or *code*). The probability _p(play | the dog loved to)_ is much higher than for most other words. Such a model can be implemented as a machine learning model, typically a neural network that can take into account a long context, and be trained over large amounts of text. Language models that have a large number of internal trainable parameters can learn complicated regularities of natural language, by learning to accurately predict the next word in a sequence.


For instance, [ELMo](https://allennlp.org/elmo) (released early 2018) uses recurrent neural networks (LSTM) for learning contextualized word vectors through language modeling.

<img src="figs/elmo-language-modeling.png" width="400" alt="Source: https://jalammar.github.io/illustrated-bert/"/>



- ELMo holds internal vector representations as it traverses the word sequence, which can be aggregated to:
    - Contextualized word vectors
    - A sentence vector, reflecting composite meaning

<img src="figs/elmo-embedding.png" width="550" alt="Source: https://jalammar.github.io/illustrated-bert/"/>

In [ ]:
#!pip3 install tensorflow_hub tensorflow

import tensorflow_hub as hub
import tensorflow as tf

# Load ELMo model (takes a little while)
elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

def elmo_vectors(sents):
    embeddings = elmo(sents, signature="default", as_dict=True)["elmo"]
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        return sess.run(embeddings)
        #sess.run(tf.tables_initializer())
        # return average of ELMo features as sentence vector
        #return sess.run(tf.reduce_mean(embeddings,1))

In [ ]:
# Get contextualized vectors for target word in different sentences

sents = """He wrote the play himself .
He did play with the kids .
His excuse didn't play well .
I saw the play with him .
He can play it by ear .""".split('\n')

target = 'play'

elmo_vecs = elmo_vectors(sents)
word_vecs = []
for i, sent in enumerate(sents):
    word_vecs.append(elmo_vecs[i][sent.split().index(target)])
    print("Sentence:", sent)
    print("Vector for '%s':" % target, word_vecs[-1])
    print()

print("Word vector size:", word_vecs[0].shape)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

vec_size = word_vecs[0].shape[0]
print("Similarities between '%s' vector in sentences:" % target)
for i in range(1, len(sents)):
    print("Sent 0-%d:" % i, cosine_similarity(word_vecs[0].reshape((1,vec_size)), 
                                              word_vecs[i].reshape((1,vec_size)))[0][0])

### Even bigger pre-trained models: BERT etc.

- In late 2018, a new, much larger pre-trained language model was released called *BERT* (Bidirectional Encoder Representations from Transformers)
- Transformers can be better parallelized than recurrent network architectures
    - --> Enables training of much larger models than before (on GPU/TPU)
- BERT Large model: 340M parameters ~ 24 Transformer layers x 1024 hidden units x 16 attention heads
    - Compared with ELMo (93.6M parameters), BERT Base (110M)

- Language modeling by learning to predict individual masked words
    - Tricky task that benefits from sophisticated "understanding" of sentences
    - Transformers are good at modeling long-range dependencies

- Computationally very demaning to train (compute cost ~$10k per run, trained on 3.3B words)
- But, fine-tuning model for downstream task (e.g., classification) is quick
- BERT was able to beat the best performing models in a range of NLP tasks at once

Simple BERT model illustration. Each token position is connected to each position in the next layer.
<img src="figs/bert.png" width="400"/>

Visualization of token-token activations for a sentence per layer and attention head.
<img src="figs/bertvis.png" source="Jesse Vig"/>

- BERT has been particularly successful at *natural language understanding (NLU)* tasks, e.g.:
    - Sentence acceptability
    - Sentiment
    - Paraphrase identification
    - Sentence similarity
    - Textual inference
    - Question answering
    

- Note, however, these models risk learning spurious statistical patterns, rather than real inference (cf. [[1]](https://www.technologyreview.com/s/615126/ai-common-sense-reads-human-language-ai2/)[[2]](https://www.aclweb.org/anthology/P19-1459.pdf))!

BERT has inspired a never-ending stream of successors. See [GLUE (NLU tasks) leaderboard](https://gluebenchmark.com/leaderboard).
<img alt="Map of the BERTology landscape" src="figs/bertology_map.png"/>


Getting started with Transformer-based language models (GPU highly recommended): [huggingface transformers library](https://huggingface.co/transformers/pretrained_models.html)

<code>import torch
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
...</code>

or with `bert-base-finnish-cased-v1` model from TurkuNLP!
